In [1]:
#svm
import menpo.io as mio
import os
from sklearn import svm
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
import math
import Models
from Models import ChangeVector, ImageData
path_to_svm_training_database = '/Programing/GR/Code/CK+/aam-images/**/**/**/*'


def process(image, crop_proportion=0.2, max_diagonal=400):
    if image.n_channels == 3:
        image = image.as_greyscale()
    image = image.crop_to_landmarks_proportion(crop_proportion)
    d = image.diagonal()
    if d > max_diagonal:
        image = image.rescale(float(max_diagonal) / d)
    return image

#process changingVector, reduce dimension from 2x68 to 1x68, by process PCA
def pca(changeVector):
    X = np.array(changeVector)
    pca_model = PCA(n_components=1)
    return pca_model.fit_transform(X)

def landmark_normalize(landmark):
    vector_max = 0
    for p in landmark:
        if(math.sqrt(float(p[0])*float(p[0]) + float(p[1])*float(p[1])) > vector_max):
            vector_max = math.sqrt(p[0]*p[0] + p[1]*p[1])
    for p in landmark:
        p[0] = p[0]/float(vector_max)
        p[1] = p[1]/float(vector_max)
    return landmark


training_images = mio.import_images(path_to_svm_training_database, verbose=True)
training_images = training_images.map(process)

path_to_facs = '/Programing/GR/Code/CK+/FACS/'
path_to_emotions = '/Programing/GR/Code/CK+/Emotion/'

#create training data
labeled_subject = []
emotion_subject = os.listdir(path_to_emotions)
for subject in emotion_subject:
    session = os.listdir(path_to_emotions + "/" + subject)
    for s in session:
        labeled_subject.append(subject + "-" + s)

count = 0;
svm_training_data = []
while(count < len(training_images)):
    file_path = str(training_images[count].path).split("\\")
    facs_path = path_to_facs + file_path[6] + '/' + file_path[7]
    gt_emotion = -1
    
    #get emotion label from Emotion Folder
    emotion_checker = file_path[6] + "-" + file_path[7]
    if(emotion_checker in  labeled_subject):
        emotion_path = path_to_emotions + file_path[6] + '/' + file_path[7]
        if(len(os.listdir(emotion_path)) != 0):
            emotion_path = emotion_path + '/' + os.listdir(emotion_path)[0]
            fi = open(emotion_path)
            for line in fi:
                if(line.split()):
                    gt_emotion = int(float(line.split()[0]))
#                 print(emotion_path + ":" + str(gt_emotion))
            fi.close()
    
    #get facs from FACS folder
    facs_path = facs_path + '/' + os.listdir(facs_path)[0]
    fi = open(facs_path, 'r')
    data_facs = {}
    tmp = []
    for line in fi: # read rest of lines
        if(line.split()):
            tmp.append(line.split())
    for f in tmp:
        data_facs[str(int(float(f[0])))] = int(float(f[1]))
    fi.close()
    
#     landmarkChange = []
#     landmark_neutral = training_images[count].landmarks['PTS'].lms.points
#     landmark_perk = training_images[count + 1].landmarks['PTS'].lms.points
#     for i in range(0,68):
#         landmarkChange.append([landmark_perk[i][0] - landmark_neutral[i][0], landmark_perk[i][1] - landmark_neutral[i][1]])

#     svm_training_data.append(ChangeVector(data_facs, landmarkChange, gt_emotion))   

    svm_training_data.append(ImageData(data_facs, training_images[count + 1].landmarks['PTS'].lms.points, gt_emotion))
    count = count + 2
    

facs = []
#create facs array
for data in svm_training_data:
    for facs_code in data.facs:
        if((int(facs_code)) not in facs):
            facs.append(int(facs_code))
facs.sort()


Found 754 assets, index the returned LazyList to import.


In [3]:
for au in svm_training_data[0].facs:
    print(svm_training_data[0].facs[au])

4
2


In [4]:
for data in svm_training_data:
    if('21' in data.facs):
        print(svm_training_data.index(data))

1
53
241


In [2]:
#create model for each action unit in facs[]
models = []
au_models_score = []
for au in facs:
    x_training = []
    y_label = []
    #create label array
    for data in svm_training_data:
        if(str(au) in data.facs):
#             y_label.append(data.facs[str(au)])
            y_label.append(1)
        else:
            y_label.append(-1)
        #create training data: 1x68 array, result of PCA process
        vector = []
        for tmp in data.landmark:
            vector.append(tmp[0])
            vector.append(tmp[1])
        x_training.append(vector)
    clf = svm.SVC(kernel='linear', decision_function_shape = 'ovr')
    clf.fit(normalize(x_training, norm='max', axis=0), y_label)
    au_models_score.append(clf.score(normalize(x_training, norm='max', axis=0), y_label))
    models.append(clf)
    
#create testing data
svm_testing_data = []
path_to_svm_testing_database = "/Programing/GR/Code/CK+/test-aam-images/**/**/**/*"
testing_images = mio.import_images(path_to_svm_testing_database, verbose=True)
testing_images = testing_images.map(process)

count = 0;
while(count < len(testing_images)):
    file_path = str(testing_images[count].path).split("\\")
#     print(file_path)
    facs_path = path_to_facs + file_path[6] + '/' + file_path[7]
    gt_emotion = -1
    
    #get emotion label from Emotion Folder
    emotion_checker = file_path[6] + "-" + file_path[7]
    if(emotion_checker in  labeled_subject):
        emotion_path = path_to_emotions + file_path[6] + '/' + file_path[7]
        if(len(os.listdir(emotion_path)) != 0):
            emotion_path = emotion_path + '/' + os.listdir(emotion_path)[0]
            fi = open(emotion_path)
            for line in fi:
                if(line.split()):
                    gt_emotion = int(float(line.split()[0]))
#                 print(emotion_path + ":" + str(gt_emotion))
            fi.close()
    
    
    #get facs from FACS folder
    facs_path = facs_path + '/' + os.listdir(facs_path)[0]
    fi = open(facs_path, 'r')
    data_facs = {}
    tmp = []
    for line in fi: # read rest of lines
        if(line.split()):
            tmp.append(line.split())
    for f in tmp:
        data_facs[str(int(float(f[0])))] = int(float(f[1]))
    fi.close()
    
#     landmarkChange = []
#     landmark_neutral = testing_images[count].landmarks['PTS'].lms.points
#     landmark_perk = testing_images[count + 1].landmarks['PTS'].lms.points
#     for i in range(0,68):
#         landmarkChange.append([landmark_perk[i][0] - landmark_neutral[i][0], landmark_perk[i][1] - landmark_neutral[i][1]])
    
#     svm_testing_data.append(ChangeVector(data_facs, landmarkChange, gt_emotion))   
    svm_testing_data.append(ImageData(data_facs, training_images[count + 1].landmarks['PTS'].lms.points, gt_emotion))
    count = count + 2

    
#evaluate trained model with test data and get score
print('#######')
print('Score: ')
au_test_score = []
for au in facs:
    x_training = []
    y_label = []
    #create label array
    for data in svm_testing_data:
        if(str(au) in data.facs):
#             y_label.append(data.facs[str(au)])
            y_label.append(1)
        else:
            y_label.append(-1)
        #create training data: 1x68 array, result of PCA process
        vector = []
        for tmp in data.landmark:
            vector.append(tmp[0])
            vector.append(tmp[1])
        x_training.append(vector)
#     print(y_label)
    au_test_score.append(models[facs.index(au)].score(normalize(x_training, norm='max', axis=0), y_label))
print(au_test_score)
print(np.mean(au_test_score))

Found 58 assets, index the returned LazyList to import.
#######
Score: 
[0.6551724137931034, 0.7931034482758621, 0.3448275862068966, 0.6206896551724138, 0.7586206896551724, 0.6551724137931034, 0.7586206896551724, 0.9655172413793104, 1.0, 0.7241379310344828, 1.0, 0.4482758620689655, 0.7241379310344828, 0.9310344827586207, 0.5172413793103449, 1.0, 0.8275862068965517, 1.0, 1.0, 0.7586206896551724, 0.9655172413793104, 0.6206896551724138, 0.896551724137931, 0.9310344827586207, 1.0, 0.9655172413793104, 1.0, 1.0, 0.8620689655172413, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
0.853448275862069


In [55]:
print(au_models_score)
print(np.mean(au_models_score))

[0.8567639257294429, 0.9549071618037135, 0.8222811671087533, 0.9363395225464191, 0.9018567639257294, 0.8514588859416445, 0.9416445623342176, 0.9655172413793104, 0.9442970822281167, 0.9310344827586207, 0.9946949602122016, 0.9655172413793104, 0.8541114058355438, 0.9708222811671088, 0.8567639257294429, 0.9787798408488063, 0.870026525198939, 0.9920424403183024, 0.9973474801061007, 0.9045092838196287, 0.9257294429708223, 0.9045092838196287, 0.9018567639257294, 0.9734748010610079, 0.9946949602122016, 0.9973474801061007, 0.9973474801061007, 0.9973474801061007, 0.9575596816976127, 0.9628647214854111, 0.9814323607427056, 0.9973474801061007, 0.9628647214854111, 0.9973474801061007, 0.9973474801061007, 0.9946949602122016]
0.9454022988505746


In [17]:
print(clf)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [36]:
print(len(svm_testing_data))

29


In [56]:
#regresssion model
wrong_predict = 0
au_score = []
for data in svm_testing_data:
    print('#####')
    local_wrong_predict = 0
    local_accurate_predict = 0
    tmp = []
    predict = []
    
    for vector in data.landmark:
        tmp.append(vector[0])
        tmp.append(vector[1])
        
    for model in models:
        if(model.predict([tmp]) != -1):
#             predict.append([facs[models.index(model)],data.facs[str(facs[models.index(model)])] , model.predict([tmp])])
            predict.append([models.index(model), model.predict([tmp])[0]])
            #print(facs[models.index(model)])
            if(str(facs[models.index(model)]) not in data.facs):
                local_wrong_predict += 1
            else: 
                local_accurate_predict += 1
        else:
            if(str(facs[models.index(model)]) in data.facs):
                local_wrong_predict += 1
    print(predict)
#     print(local_accurate_predict)
    au_score.append(float(local_accurate_predict)/float(len(data.facs)))
    print("---")
    print(data.facs)
    wrong_predict += local_wrong_predict
    
# print(wrong_predict)
# print(sum(au_score)/float(len(au_score)))

#####
[[0, 1], [1, 1], [3, 1], [6, 1], [7, 1], [8, 1], [9, 1], [13, 1], [17, 1], [21, 1], [24, 1], [25, 1], [31, 1], [32, 1], [33, 1], [34, 1]]
---
{'14': 2, '24': 2, '12': 2}
#####
[[0, 1], [1, 1], [3, 1], [8, 1], [9, 1], [10, 1], [13, 1], [17, 1], [21, 1], [24, 1], [25, 1], [31, 1], [32, 1], [33, 1], [34, 1]]
---
{'14': 2, '12': 1}
#####
[[0, 1], [1, 1], [3, 1], [8, 1], [10, 1], [13, 1], [17, 1], [21, 1], [23, 1], [24, 1], [25, 1], [30, 1], [31, 1], [32, 1], [33, 1], [34, 1]]
---
{'14': 0}
#####
[[0, 1], [1, 1], [3, 1], [7, 1], [8, 1], [9, 1], [10, 1], [13, 1], [17, 1], [21, 1], [24, 1], [25, 1], [30, 1], [31, 1], [32, 1], [33, 1], [34, 1]]
---
{'14': 2, '12': 1}
#####
[[0, 1], [1, 1], [3, 1], [10, 1], [13, 1], [17, 1], [21, 1], [24, 1], [25, 1], [30, 1], [31, 1], [32, 1], [33, 1], [34, 1]]
---
{'14': 2, '17': 3}
#####
[[0, 1], [1, 1], [3, 1], [6, 1], [8, 1], [9, 1], [10, 1], [13, 1], [17, 1], [21, 1], [24, 1], [25, 1], [30, 1], [31, 1], [32, 1], [33, 1], [34, 1]]
---
{'14': 2, '17':

In [43]:
#get predict result
y_label = []
for au in facs:
    x_training = []
    y_label_tmp = []
    #create label array
    for data in svm_testing_data:
        if(str(au) in data.facs):
#             if(data.facs[str(au)] == 0):
#                 y_label.append(2)
#             else:
            y_label_tmp.append(data.facs[str(au)])
#             y_label.append(1)
        else:
            y_label_tmp.append(-1)
        #create training data: 1x68 array, result of PCA process
        vector = []
        for tmp in data.landmark:
            vector.append(tmp[0])
            vector.append(tmp[1])
        x_training.append(vector)
    y_label.append(y_label_tmp)
for model in models:
    print("=====")
    print(model.predict(x_training))
    print("==")
    print(y_label[models.index(model)])

=====
[-1  2  2  2  2  2  2  2  2  2 -1 -1  2  2  2  2  2  2  2  2  2  2  2  2
  2  2 -1 -1  2]
==
[-1, -1, -1, -1, -1, -1, 2, -1, 2, 2, -1, -1, 3, -1, -1, -1, -1, -1, 2, 1, -1, 2, -1, -1, 4, 4, -1, -1, 2]
=====
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
==
[-1, -1, -1, -1, -1, -1, 2, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 2, 1, -1, -1, -1, -1, 4, 2, -1, -1, 1]
=====
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1]
==
[-1, -1, -1, -1, -1, -1, 1, 3, 3, 4, 3, -1, 4, 4, -1, 3, 2, -1, 2, 2, -1, 2, -1, -1, 2, 3, -1, 4, 4]
=====
[2 0 0 2 2 0 0 2 2 2 2 2 2 0 2 2 2 2 0 2 2 0 2 2 0 2 2 2 2]
==
[-1, -1, -1, -1, -1, -1, -1, 1, 4, -1, 3, -1, 3, 2, -1, -1, 3, -1, 4, -1, -1, -1, 2, -1, 4, -1, -1, 1, 3]
=====
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1]
==
[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 2, -1, -1, -1, -1, -1, -1]
=====
[5 5 5 5 5 5 5 5 5

In [3]:
#build rule-based system
#emotion class have name = str, criteria = function, facs required = [[]], caculate score of input
class Emotion:
    def __init__(self, name, facs_required, criteria):
        self.name = name
        self.facs_required = facs_required
        self.criteria = criteria
    
    def criteria(self, facs_input):
        return True
    
    def score(self,facs_input = []):
        if(self.criteria(facs_input) == True):
            max = 0
            for required in self.facs_required:
                au_count = 0
                for facs in facs_input:
                    if facs in required:
                        au_count += 1
                if au_count/float(len(required)) >= max:
                    max = au_count/float(len(required))
            return max
        else:
            return 0
    
def angry_criteria(facs_input):
    if(23 in facs_input):
        return True
    return False

def disgus_criteria(facs_input):
    if(9 in facs_input or 10 in facs_input):
        return True
    return False

def fear_criteria(facs_input):
    if(1 in facs_input and 2 in facs_input and 3 in facs_input):
        return True
    return False

def surprise_criteria(facs_input):
    if(1 in facs_input and 2 in facs_input):
        return True
    if(5 in facs_input):
        return True
    return False

def sadness_criteria(facs_input):
    return True

def happy_criteria(facs_input):
    if(12 in facs_input):
        return True
    return False

def contempt_criteria(facs_input):
    if(14 in facs_input):
        return True
    return False

happy = Emotion('happy', [[6,12]], happy_criteria)
sadness = Emotion('sadness', [[1,4,5], [6,15], [1,4,15]], sadness_criteria)
surprise = Emotion('surprise', [[1,2,5,26]], surprise_criteria)
fear = Emotion('fear', [[1,2,4,5,7,20,26]], fear_criteria)
angry = Emotion('angry', [[4,5,7,23]], angry_criteria)
disgust = Emotion('disgust', [[9,15,16], [10,15,16]], disgus_criteria)
contempt = Emotion('contempt', [[12,14]], contempt_criteria)

emotions = [happy, sadness, surprise, fear, angry, disgust, contempt]



In [3]:
#emotion prediction model -> this is the shit
result = []
x_training = []
y_label = []
for data in svm_training_data:
    if(data.emotion != -1):
        tmp = []
        for vector in data.landmark:
            tmp.append(vector[0])
            tmp.append(vector[1])
        x_training.append(tmp)
        y_label.append(data.emotion)
    
clf = svm.LinearSVC()
clf.fit(normalize(x_training, norm='max', axis=0), y_label)
print(clf.score(normalize(x_training, norm='max', axis=0), y_label))

x_training = []
y_label = []
for data in svm_testing_data:
    if(data.emotion != -1):
        tmp = []
        for vector in data.landmark:
            tmp.append(vector[0])
            tmp.append(vector[1])
        x_training.append(tmp)
        y_label.append(data.emotion)

print(clf.predict(normalize(x_training, norm='max', axis=0)))
print(y_label)
print(clf.score(normalize(x_training, norm='max', axis=0), y_label))

0.9184782608695652
[3 5 7 1 1 3 5 4 4 4 1 3 5 7 6 1 3 5 7 4 4 4 1 3 7 6 3 3 5]
[2, 2, 2, 2, 2, 2, 2, 1, 4, 6, 1, 2, 4, 1, 2, 6, 1, 2, 4, 6, 2, 6, 1, 2, 4, 6, 2, 1, 4]
0.13793103448275862


In [6]:
#save model
import pickle
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)
        
save_object(models, '/Programing/GR/Code/Python/models/au_models.pkl')
#save_object(facs, '/Programing/GR/Code/Python/models/facs.pkl')

In [23]:
#create json log file
import json
log_path = '/Programing/GR/Code/Python/log/'
import time
import json
ts = int(time.time())
#jsonAdder = json.dumps()
output = {'n_train': len(training_images), 'n_test': len(testing_images), 'au_score' : au_models_score, 'facs': facs }
#print json.dumps(output, sort_keys=True, indent=4, separators=(',', ': '))
with open(log_path + 'system_log' + str(ts) + '.txt', "w+") as outfile:
    json.dump(output, outfile,sort_keys=True, indent=4, separators=(',', ': '))